In [1]:
import os
import shutil
import tarfile
import requests
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
DATASET_ID   = '1cAJdvAZDolurN3KCZcYz_YJSMV-aIzWT'

# -------- TEST USER ----------- #

TEST_USER      = '001'

# -------------BASE DIR (MODIFY THIS TO YOUR NEED) ------------ #
BASE_DIR     = '../'

DATA_DIR     = 'Sensor-Data/'

USERS        = ['001', '002', '003', '004', '005', '006', '007']
# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES     = ['deaf', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

BATCH_SIZE     = 32

# ------------- FOR THE GREATER GOOD :) ------------- #
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 25
LEARNING_RATE  = 0.001

In [3]:
#--------------------- Download util for Google Drive ------------------- #

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
        
    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
        
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

def download_data(fid, destination):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(destination)
        print('√')
    except:
        print('✕')
        
    print('creating data directory ... ', end='')
    os.mkdir(destination)
    print('√')
    
    print('downloading dataset from the repository ... ', end='')
    filename = os.path.join(destination, 'dataset.tar.xz')
    try:
        download_file_from_google_drive(fid, filename)
        print('√')
    except:
        print('✕')
        
    print('extracting the dataset ... ', end='')
    try:
        tar = tarfile.open(filename)
        tar.extractall(destination)
        tar.close()
        print('√')
    except:
        print('✕')   

In [4]:
# ------- Comment This if already downloaded -------- #

# destination = os.path.join(BASE_DIR, DATA_DIR)
# download_data(DATASET_ID, destination)

In [16]:
def load_data():
    X_train = np.zeros((TRAIN_LEN, 9, 1))
    X_test = np.zeros((TEST_LEN, 9, 1))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
        
    for user in USERS:
        print('loading data for user ' + user + ' ... ', end='')
        path = os.path.join(BASE_DIR, DATA_DIR, user)
        for gesture in GESTURES:
            filename = gesture + '.csv'
            file_path = os.path.join(path, filename)
            f1 = np.median(pd.read_csv(file_path)['flex_1'].to_numpy().reshape(-1, 150), axis=1, keepdims=True)
            f2 = np.median(pd.read_csv(file_path)['flex_2'].to_numpy().reshape(-1, 150), axis=1, keepdims=True)
            f3 = np.median(pd.read_csv(file_path)['flex_3'].to_numpy().reshape(-1, 150), axis=1, keepdims=True)
            f4 = np.median(pd.read_csv(file_path)['flex_4'].to_numpy().reshape(-1, 150), axis=1, keepdims=True)
            f5 = np.median(pd.read_csv(file_path)['flex_5'].to_numpy().reshape(-1, 150), axis=1, keepdims=True)
            qw = np.median(pd.read_csv(file_path)['Qw'].to_numpy().reshape(-1, 150), axis=1, keepdims=True)
            qx = np.median(pd.read_csv(file_path)['Qx'].to_numpy().reshape(-1, 150), axis=1, keepdims=True)
            qy = np.median(pd.read_csv(file_path)['Qy'].to_numpy().reshape(-1, 150), axis=1, keepdims=True)
            qz = np.median(pd.read_csv(file_path)['Qz'].to_numpy().reshape(-1, 150), axis=1, keepdims=True)
            flex_data = np.concatenate([f1, f2, f3, f4, f5, qw, qx, qy, qz], axis=1)
            
            if user != TEST_USER:
                X_train[train_count: train_count + 10, :] = np.expand_dims(flex_data, axis=-1)
                y_train[train_count: train_count + 10, 0] = GESTURES.index(gesture)
                train_count = train_count + 10
                
            else:
                X_test[test_count: test_count + 10, :] = np.expand_dims(flex_data, axis=-1)
                y_test[test_count: test_count + 10, 0] = GESTURES.index(gesture)
                test_count = test_count + 10
                
        print('√')
        
    return X_train, X_test, y_train, y_test

In [17]:
X_train, X_test, y_train, y_test = load_data()

loading data for user 001 ... √
loading data for user 002 ... √
loading data for user 003 ... √
loading data for user 004 ... √
loading data for user 005 ... √
loading data for user 006 ... √
loading data for user 007 ... √


In [18]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization(input_shape=(9, 1))
normalizer.adapt(X_train)

In [19]:
model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Flatten(input_shape=(8, 1)),
    tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 9, 1)              3         
_________________________________________________________________
flatten (Flatten)            (None, 9)                 0         
_________________________________________________________________
dense (Dense)                (None, 32)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [21]:
history = model.fit(X_train, y_train, epochs=EPOCHS)

Epoch 1/25
30/30 [==============================] - 0s 1ms/step - loss: 2.8067 - accuracy: 0.0924
Epoch 2/25
30/30 [==============================] - 0s 1ms/step - loss: 2.6233 - accuracy: 0.1017
Epoch 3/25
30/30 [==============================] - 0s 1ms/step - loss: 2.4849 - accuracy: 0.1651
Epoch 4/25
30/30 [==============================] - 0s 1ms/step - loss: 2.2377 - accuracy: 0.2470
Epoch 5/25
30/30 [==============================] - 0s 1ms/step - loss: 1.9196 - accuracy: 0.3144
Epoch 6/25
30/30 [==============================] - 0s 1ms/step - loss: 1.7495 - accuracy: 0.3620
Epoch 7/25
30/30 [==============================] - 0s 1ms/step - loss: 1.5723 - accuracy: 0.4143
Epoch 8/25
30/30 [==============================] - 0s 2ms/step - loss: 1.5129 - accuracy: 0.4065
Epoch 9/25
30/30 [==============================] - 0s 1ms/step - loss: 1.4334 - accuracy: 0.4485
Epoch 10/25
30/30 [==============================] - 0s 1ms/step - loss: 1.3897 - accuracy: 0.4495
Epoch 11/25
30/30 [

In [22]:
prob = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
y_pred = prob.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test.ravel(), y_pred))

              precision    recall  f1-score   support

         0.0       0.40      1.00      0.57        20
         1.0       0.00      0.00      0.00        10
         2.0       0.15      0.30      0.20        10
         4.0       0.00      0.00      0.00        10
         5.0       0.00      0.00      0.00        10
         6.0       0.03      0.10      0.05        10
         7.0       1.00      1.00      1.00        10
         8.0       0.00      0.00      0.00        10
         9.0       0.00      0.00      0.00        10
        10.0       1.00      1.00      1.00        10
        11.0       0.00      0.00      0.00        10
        12.0       0.91      1.00      0.95        10
        13.0       0.00      0.00      0.00        10
        14.0       0.53      1.00      0.69        10
        15.0       0.00      0.00      0.00        10

    accuracy                           0.40       160
   macro avg       0.27      0.36      0.30       160
weighted avg       0.28   

/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andromeda/Ai/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
